In [1]:
!pip install -q twarc #Twarc
!pip install -q tweepy # Tweepy 3.8.0
!pip install -q argparse #Argparse 3.2
!pip install -q xtract #Xtract 0.1 a3
!pip install -q wget #Wget 3.2

     |████████████████████████████████| 54 kB 894 kB/s 
     |████████████████████████████████| 90 kB 3.7 MB/s 


In [2]:
import json
import tweepy
from tweepy import OAuthHandler
import glob
import pandas as pd
import numpy as np
import os
import re
from zipfile import ZipFile
from google.colab import files
from google.colab import drive
from tqdm import tqdm
tqdm.pandas()
drive.mount('/content/drive' ,force_remount=True)
os.listdir()#!pwd

Mounted at /content/drive


['.config', 'drive', 'sample_data']

In [3]:
os.chdir('drive/My Drive/vaccine')
!pwd


/content/drive/My Drive/vaccine


In [4]:
# os.listdir('drive/My Drive/vaccine/shuffled/hydrated_tweet_id')
# list_of_files[0]
for file in glob.glob('shuffled/hydrated_tweet_id/*.csv'):
  print (file)

shuffled/hydrated_tweet_id/_201213_201219.csv
shuffled/hydrated_tweet_id/_210221_210227.csv
shuffled/hydrated_tweet_id/_170219_170225.csv
shuffled/hydrated_tweet_id/_170514_170520.csv
shuffled/hydrated_tweet_id/_170625_170701.csv
shuffled/hydrated_tweet_id/_170806_170812.csv
shuffled/hydrated_tweet_id/_170917_170923.csv
shuffled/hydrated_tweet_id/_171029_171104.csv
shuffled/hydrated_tweet_id/_171210_171216.csv
shuffled/hydrated_tweet_id/_180304_180310.csv
shuffled/hydrated_tweet_id/_180415_180421.csv
shuffled/hydrated_tweet_id/_180708_180714.csv
shuffled/hydrated_tweet_id/_181014_181020.csv
shuffled/hydrated_tweet_id/_181125_181201.csv
shuffled/hydrated_tweet_id/_190106_190112.csv
shuffled/hydrated_tweet_id/_190331_190406.csv
shuffled/hydrated_tweet_id/_190512_190518.csv
shuffled/hydrated_tweet_id/_190623_190629.csv
shuffled/hydrated_tweet_id/_190804_190810.csv
shuffled/hydrated_tweet_id/_190915_190921.csv
shuffled/hydrated_tweet_id/_191027_191102.csv
shuffled/hydrated_tweet_id/_191208

In [32]:
def rename_tweet_id2(file): 
  df = pd.read_csv(file, dtype={'Unnamed: 0':'Int64', "tweet_id":'Int64', "tweet_id2":'Int64',"tweet_id3":'Int64',"tweet_id4":'Int64'})
  if len(df.columns) >3:
    # try:
      df.rename(columns={'tweet_id':'used_train', "tweet_id2":'tweet_id'}, inplace = True)
      df.drop(['Unnamed: 0'], axis =1, inplace = True)
      # df['used_train'] = df['used_train'].astype('int64')

      df['tweet_id'] = df['tweet_id'].astype('Int64')
      # df['tweet_id3'] = df['tweet_id3'].astype('int64')
      # df['tweet_id4'] = df['tweet_id4'].astype('int643')
      df.to_csv(file, index = False )
    # except:
    #   print('column already renamed')

In [30]:
filepath = 'shuffled/hydrated_tweet_id/test201213_201219.csv'

df = rename_tweet_id2(filepath)
df= pd.read_csv(filepath)
df.head()

,used_train,tweet_id,tweet_id3,tweet_id4
0,1340087769125806080,1.337914e+18,1.339100e+18,1.339821e+18
1,1338881220344745985,1.339284e+18,1.340272e+18,1.338882e+18
2,1339089316706971648,1.338919e+18,1.338019e+18,1.339307e+18
3,1338858437195649028,1.338174e+18,1.338810e+18,1.340017e+18
4,1338714457258614784,1.339636e+18,1.339750e+18,1.339453e+18


In [36]:
!python get_metadata.py -i {'shuffled/hydrated_tweet_id/test201213_201219.csv'} -o {'hydrated/hydrated_testcase201213_201219'} -k api_keys.json

Your twitter api credentials are valid.
hydrated/hydrated_testcase201213_201219
total ids: 451169
metadata collection complete
creating master json file
currently getting 0 - 100
currently getting 100 - 200
currently getting 200 - 300
currently getting 300 - 400
currently getting 400 - 500
currently getting 500 - 600
currently getting 600 - 700
currently getting 700 - 800
currently getting 800 - 900
currently getting 900 - 1000
currently getting 1000 - 1100
currently getting 1100 - 1200
currently getting 1200 - 1300
currently getting 1300 - 1400
currently getting 1400 - 1500
currently getting 1500 - 1600
currently getting 1600 - 1700
currently getting 1700 - 1800
currently getting 1800 - 1900
currently getting 1900 - 2000
currently getting 2000 - 2100
currently getting 2100 - 2200
currently getting 2200 - 2300
currently getting 2300 - 2400
currently getting 2400 - 2500
currently getting 2500 - 2600
currently getting 2600 - 2700
currently getting 2700 - 2800
currently getting 2800 - 290

In [33]:
def hydrate_csv(file_name):
  output_name = "hydrated/hydrated_tweets_test_" + file_name[-17:-4]
  !python get_metadata.py -i {file_name} -o {output_name} -k api_keys.json
  print(file_name, 'shuffled/hydrated_tweet_id/'+file_name[-17:])
  os.rename(file_name ,'shuffled/hydrated_tweet_id/test'+file_name[-17:] )
  return output_name
# hydrated_filepath = hydrate_csv(file)

#this func unzips all of the json tweets that the intitial unroll creates.

def unzip_move(filepath):
  for   i,file in enumerate(glob.glob("*.zip")):
 
    if os.path.getsize(file) > 0:
      with ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(f'/content/drive/My Drive/vaccine/shuffled/hydrated/{i}unzipped/')
        for unzipped_file in glob.glob(f'/content/drive/My Drive/vaccine/shuffled/hydrated/{i}unzipped/shuffled/hydrated/*.csv'):
          os.rename(unzipped_file,'/content/drive/My Drive/vaccine/shuffled2/' +unzipped_file[80:] )
        
def unrolled(dict_to_of_dicts):
  
  list_ofdicts = dict_to_of_dicts["hashtags"]
  if len(list_ofdicts) >0:
    return list_ofdicts[0]['text']
  else:
    return ''

def clean_tweet(text):
    '''
    clean tweets by removing usernames and urls
    [text] : str
    '''

    without_usernames = []
    for s in text.split():
        if '@' not in s:
            without_usernames.append(s)
    text = ' '.join(without_usernames)

    without_url = []
    for s in text.split():
        if 'http' not in s:
            without_url.append(s)
    text = ' '.join(without_url)

    without_newline =[]
    for s in text.split():
      if '\n'not in s:
        without_newline.append(s)
    text = ' '.join(without_url)

    text = text.strip()

    without_hashtags = []
    for s in text.split():
      if '#' not in s:
          without_hashtags.append(s)
      else:
        without_hashtags.append(s[1:])
    text = ' '.join(without_hashtags)

    text = text.strip()
    return text 


def tweet_cleaner(file):
    df = pd.read_json(file, orient = 'records', lines = True )
    #remove retweets to keep only unique
    df = df[pd.isnull(df['retweeted_status']) ]
    # keep only date created, unique tweet id, text of tweet and entities to extract hashtags from
    df = df[['created_at','id','text',  'entities']]
    df['text'] = df['text'].progress_apply(clean_tweet)
    
    # df['hashtags'] = df['entities'].progress_apply(lambda idx: unrolled(idx))
    # df.drop(columns= 'entities', inplace=True)
    df['word_count'] = df['text'].progress_apply(lambda x: len(x.split(sep =" ")))  
        
    df.to_csv("shuffled2/trimmed json files/trimmed_" + file[-18:]+'.csv' , index = False)

    if os.path.exists(file):
      os.remove(file)
      print(f"{file} cleaned, trimmed,exported and removed")
    else:
      print("The file does not exist")

In [34]:
CONSUMER_KEY = "vu1RUkGEZUjHaTKF4hykq05wD" #@param {type:"string"}
CONSUMER_SECRET_KEY = "RtSwhg5FcjMJL9uOg1b84tQvSGiIGdmQHqecx0totJgBdWBkoF" #@param {type:"string"}
ACCESS_TOKEN_KEY = "1449439777258975232-O3juU6NYDfUub4kDVfVnyQdfRK2jge" #@param {type:"string"}
ACCESS_TOKEN_SECRET_KEY = "oRnh4NL8D5XoBz2oiuU5nBDXzVoIDWIcBT3L728YItOes" #@param {type:"string"}
with open('api_keys.json', 'w') as outfile:
    json.dump({
    "consumer_key":CONSUMER_KEY,
    "consumer_secret":CONSUMER_SECRET_KEY,
    "access_token":ACCESS_TOKEN_KEY,
    "access_token_secret": ACCESS_TOKEN_SECRET_KEY
     }, outfile)

In [35]:
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET_KEY)

api = tweepy.API(auth, wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)
cant_clean_file =[]

In [10]:
filepath = 'shuffled/hydrated_tweet_id/test201213_201219.csv'
df= pd.read_csv(filepath)
df.columns

Index(['Unnamed: 0', 'tweet_id', 'tweet_id2', 'tweet_id3', 'tweet_id4'], dtype='object')

In [ ]:

# print(filepath)
for i,file in enumerate(glob.glob('shuffled/hydrated_tweet_id/*.csv')):
  if 'test' not in file:
#     print('true')
    rename_tweet_id2(file)
    # output_name = "hydrated/hydrated_tweets_test_" + filepath[-17:-4]
    # !python get_metadata.py -i {filepath} -o {output_name} -k api_keys.json
    # print(file_name, 'shuffled/hydrated_tweet_id/'+file_name[-17:])
    # os.rename(filepath ,'shuffled/hydrated_tweet_id/test'+filepath[-17:] )
    output_name = hydrate_csv(file)
    os.remove(output_name +'.csv')
    # os.remove(output_name + '_short.json')
    os.remove(output_name +".zip")
    try:
      tweet_cleaner(output_name_short.json)
    except:
      print(f'could not clean {output_name}')
      # cant_clean_file.append(output_name)
    # #read in json
# df = pd.read_json(output_name, orient = 'records', lines = True )
# #remove retweets to keep only unique
# df = df[pd.isnull(df['retweeted_status']) ]
# tqdm.pandas()
# from tqdm import tqdm
# tqdm.pandas()
# df['text'] = df['text'].progress_apply(clean_tweet)
# df['hashtags'] = df['entities'].progress_apply(lambda idx: unrolled(idx))
# df.drop(columns= 'entities', inplace= True)

print(output_name)

Your twitter api credentials are valid.
hydrated/hydrated_tweets_test_170219_170225
total ids: 11834
metadata collection complete
creating master json file
currently getting 0 - 100
currently getting 100 - 200
currently getting 200 - 300
currently getting 300 - 400
currently getting 400 - 500
currently getting 500 - 600
currently getting 600 - 700
currently getting 700 - 800
currently getting 800 - 900
currently getting 900 - 1000
currently getting 1000 - 1100
currently getting 1100 - 1200
currently getting 1200 - 1300
currently getting 1300 - 1400
currently getting 1400 - 1500
currently getting 1500 - 1600
currently getting 1600 - 1700
currently getting 1700 - 1800
currently getting 1800 - 1900
currently getting 1900 - 2000
currently getting 2000 - 2100
currently getting 2100 - 2200
currently getting 2200 - 2300
currently getting 2300 - 2400
currently getting 2400 - 2500
currently getting 2500 - 2600
currently getting 2600 - 2700
currently getting 2700 - 2800
currently getting 2800 - 

In [1]:
pd.read_json('hydrated/hydrated_tweets_test_170108_170114', orient = 'records', lines = True )

NameError: ignored

In [4]:
testDf = pd.read_json( 'hydrated/hydrated_tweets_test_210404_210410_short.json',orient = 'records', lines = True)
testDf.columns
testDf[testDf['is_retweet']== False].count()

testDf.head()